# 신경망 제작

기본 신경망 학습 절차

1. 가중치를 가지고 있는 신경망을 정의 (가중치는 초기화되지 않은 임의의 수, pre_trained라면 기존의 가지고 있는 가중치로 시작)
2. data의 입력
3. 신경망의 순전파로 data를 계산한  prediction 도출
4. 사전 정의된 loss function을 통해 정답과 prediction의 loss를 계산
5. 신경망 매개변수에 변화도를 역전파함
6. 신경망의 가중치를 사전에 정의한 갱신 규칙(=optimizer)에 따라 갱신

In [24]:
# 간단한 신경망 정의

import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 3) ## image 입력 1개, 출력 6개, 커널 사이즈 3x3 정사각형 컨벌루션 행렬
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        self.fc1 = nn.Linear(16*6*6, 120 ) ## 6x6 사이즈 이미지 차원 16를  펼쳐서 120개 출력
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):

        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) ## 2x2 사이즈 커널로 conv1 후 relu 결과를 맥스 풀링, 사이즈 축소
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) ## 역시나 2x2 사이즈 커널, 정수 하나를 주면 nxn정사각형 커널 행렬로 인식
        x = x.view(-1, self.num_flat_features(x)) ## num_flat_features 함수로 펼치기
        x = F.relu(self.fc1(x)) # full connected layer 통과
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x): ## 펼쳤을 때 차원을 구하는 함수
        size = x.size()[1:] # 배치 차원을 제외한 모든 차원
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [25]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [26]:
## 학습 가능한 매개변수 숫자 반환

params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1의 가중치
print(params[2].size()) # conv2의 가중치
print(params[4].size()) # fc1의 가중치
print(params[6].size()) # fc2의 가중치
print(params[8].size()) # fc3의 가중치

10
torch.Size([6, 1, 3, 3])
torch.Size([16, 6, 3, 3])
torch.Size([120, 576])
torch.Size([84, 120])
torch.Size([10, 84])


In [27]:
## LeNet의 예상 입력 사이즈는 32x32 사이즈를 변경 시켜줘야함

input = torch.randn(1, 1, 32,32)
output = net(input)
print(output)

tensor([[-0.0398,  0.0643,  0.0542, -0.0516, -0.0764,  0.1428, -0.0488,  0.0165,
         -0.0198,  0.0341]], grad_fn=<AddmmBackward0>)


In [31]:
## 모든 매개변수 변화도 버퍼를 0으로 초기화, 무작위 변화도로 역전파

net.zero_grad()
output.backward(torch.randn(1,10))

In [28]:
## 손실함수 정의

criterion = nn.MSELoss() ## nn에서 지원하는 MSE 로스 함수

# ex 

target = torch.rand(1,10)
loss = criterion(output, target)
print(loss)

tensor(0.3228, grad_fn=<MseLossBackward0>)


In [29]:
## 역전파를 추적

print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

In [30]:
## 역전파를 위해  존재하는 가중치 버퍼를 제거, 제거하지 않으면 변화도 누적
## backward를 이미 시행을 하면 가중치를 기록했던 메모리 버퍼가 해제되어, 두번재 backward를 선언하면 runtime error를 발생, 위의 output의 backward를 하지 않음으로
## 실행 오류는 피했으나 최선인지는 모르겠음.

net.zero_grad() # 모든 매개변수의 변화도 버퍼를 0으로 만들기

print('conv1.bias.grad befort backward')
print(net.conv1.bias.grad)

loss.backward(retain_graph=True)

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad befort backward
None
conv1.bias.grad after backward
tensor([ 0.0056, -0.0007, -0.0081, -0.0237,  0.0012, -0.0122])


In [32]:
## 가중치 갱신 방법

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)


In [33]:
## optim에서 가중치 갱신 optimizer를 제공

import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01) 

# 학습 과정(training loop)에서
optimizer.zero_grad() # 변화도 버퍼 0으로 만들기
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # 갱신하기